In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import subprocess as sp
from scipy.interpolate import interpn

In [ ]:
def read_dist(filename):
    with open(filename) as f:
        #w, h = [int(x) for x in next(f).split()] # read first line
        ys = []
        for line in f: # read rest of lines
            ys.append([int(x) for x in line.split()])
        return ys

In [ ]:
def get_compress_size(dataset, compressor="sz", nprocs=1):
    '''
    calls the libpressio container via runcmd wrapper
    returns the compressed size across the given commpressor and 
    process count extracted from json
    '''
    result = sp.run(f"./runcmd data_dist.py -n {nprocs} -j {dataset} | jq '.{compressor}[].metrics[\"size:compressed_size\"]'",
          shell=True, check=True, capture_output=True, text=False)
    
    return [int(x) for x in str(result.stdout, encoding='utf-8').splitlines()]

In [ ]:
def plot_compress(ys):
    xs = [x for x in range(len(ys))]
    plt.plot(xs, ys)
    plt.show()
    # Make sure to close the plt object once done
    plt.close()

In [ ]:
def plot_hist(ys, bins=10):
    npa = np.asarray(ys, dtype=np.float32)
    plt.hist(npa, bins)
    plt.show()

In [ ]:
ys=get_compress_size(dataset="datasets/CLOUDf48.bin.f32", compressor="zfp", nprocs="100")

In [ ]:
plot_compress(ys)

In [ ]:
plot_hist(ys)

In [ ]:
ys=read_dist('sz-100-slice')

In [ ]:
plot_compress(ys)

In [ ]:
plot_hist(ys)

In [ ]:
ys=read_dist('zfp-25-slice')

In [ ]:
plot_compress(ys)

In [ ]:
plot_hist(ys)

In [ ]:
ys=read_dist('sz-25-slice')

In [ ]:
plot_compress(ys)

In [ ]:
plot_hist(ys)

## Test 1D interpolations

https://docs.scipy.org/doc/scipy/tutorial/interpolate/1D.html

In [ ]:
def gen_1d(y, orig, new):
    npy=np.array(y)
    xnew = np.linspace(0, len(npy), num=new)
    return np.interp(xnew, range(len(npy)), npy)

In [ ]:
sample=4

In [ ]:
scale=10

In [ ]:
ys=get_compress_size(dataset="datasets/CLOUDf48.bin.f32", compressor="zfp", nprocs=sample)

In [ ]:
plot_compress(ys)

In [ ]:
plot_compress(gen_1d(ys, sample, scale))

In [ ]:
plot_compress(gen_1d(ys, sample, 100))

In [ ]:
sample=100

In [ ]:
ys=get_compress_size(dataset="datasets/CLOUDf48.bin.f32", compressor="zfp", nprocs=sample)

In [ ]:
plot_compress(ys)

In [ ]:
sample

In [ ]:
plot_compress(gen_1d(ys, sample, 1000))

In [ ]:
plot_hist(ys, 50)

In [ ]:
plot_hist(gen_1d(ys, sample, 1000), 50)

## Tri-linear with scipy interpn

Documentation of [interpn()](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interpn.html#scipy.interpolate.interpn)

Code based on example from documentation

In [ ]:
ys=get_compress_size(dataset="datasets/CLOUDf48.bin.f32", compressor="zfp", nprocs="25")

In [ ]:
g_truth = get_compress_size(dataset="datasets/CLOUDf48.bin.f32", compressor="zfp", nprocs="100")

In [ ]:
ys=np.array(ys)

In [ ]:
yidx = np.array(range(len(ys)))

In [ ]:
newy = np.linspace(0, len(ys)-1, 100)

In [ ]:
genval=interpn((yidx,), ys, newy)

In [ ]:
plot_compress(ys)

In [ ]:
plot_compress(genval)

In [ ]:
plot_compress(g_truth)

In [ ]:
plot_hist(ys)

In [ ]:
plot_hist(genval)

In [ ]:
plot_hist(g_truth)